In [1]:
! pip install transformers torch ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.1 kB/s eta 0:00:00m eta 0:00:0136m0:00:05m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 130.5 kB/s eta 0:00:00m eta 0:00:010:00:28
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 134.2 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.2/481.2 kB 122.7 kB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 89.1 kB/s eta 0:00:0031m85.4 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 66.6 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.6/184.6 kB 95.1 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.9/436.9 kB 103.5 kB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 240.7 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 323.9 kB/s eta 0:00:00m eta 0:00:010

In [3]:
import torch
from transformers import pipeline, set_seed, AutoModelForCausalLM, AutoTokenizer, TrainingArguments

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import numpy as np
print(np.__version__)


1.26.4


In [28]:
import pandas as pd
keywords_df = pd.read_csv("../data/keywords_tags_titles_descriptions.csv")

In [29]:
print(keywords_df)


           Title          Tags           Description
0             12          2018                    30
1         advice        advice             adventure
2           best          best                advice
3          doing         chris                  amzn
4         future     interview                 books
5         jordan        jordan            chriswillx
6           life          life               contact
7         matter        modern               content
8        meaning    motivation              discount
9        monster  motivational                   don
10    motivation      peterson           educational
11  motivational       podcast                  free
12      negative      powerful                 going
13      peterson    productive                  idea
14      reminder         rules             instagram
15         rules          self                jordan
16       screwed        speech       jordanbpeterson
17      speeches       success  jordanpeterson

In [30]:
stop_words = ['the', 'a', 'and', 'is', 'in', 'to', 'for', 'on', 'of', 'with', 'as', 'an']
keyword_df_clean = keywords_df[~keywords_df.isin(stop_words)].dropna()


In [31]:
!pip install markovify

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18695 sha256=a5e80f670eb77090af11c8d759ec7374f42b66dea58142d32ed48c9b0cbcc5a3
  Stored in directory: /Users/arnoldm./Library/Caches/pip/wheels/9c/20/eb/1a3fb93f3132f2f9683e4efd834800f80c53aeddf50e84ae80
Successfully built markovify

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [39]:
import markovify
import pandas as pd

# Load your YouTube data
youtube_data = pd.read_csv("../data/youtube_data.csv")  # Ensure it has a 'Title' column

# Combine all titles into one string
text_data = "\n".join(youtube_data["Title"].dropna().astype(str))

# Train the Markov model
markov_model = markovify.Text(text_data, state_size=1)  # Use 1 instead of 2

# Generate a random title
print(markov_model.make_sentence())


KeyError: ('___BEGIN__',)

the data set is too small for markovify

In [44]:
titles = youtube_data["Title"].tolist()
print(titles)


['Jordan Peterson - How To Destroy Your Negative Beliefs (4K)', 'Jordan Peterson - 7 Harsh Truths To Take Control Of Your Life', 'DISCIPLINE YOURSELF -  Best Motivational Speeches by Jordan Peterson', "Jordan Peterson's Life Advice Will Change Your Future (MUST WATCH)", 'The Choice We All Have , But Only a Few Apply It | Jordan Peterson', 'FIND MEANING IN YOUR LIFE - JORDAN PETERSON [AMAZING]', '"You Should Be A Monster" | Jordan Peterson Motivation', "You're Screwed No Matter What You Do | Jordan Peterson", 'A Reminder To Treat Yourself Better | Jordan Peterson Motivation | 12 Rules For Life', 'Anything Worth Doing Is Worth Doing Badly | Jordan Peterson | Best Life Advice']


In [ ]:
import random
from collections import defaultdict

# Step 1: Build Markov Chain
def build_markov_chain(titles):
    markov_chain = defaultdict(list)
    
    for title in titles:
        words = title.split()
        for i in range(len(words) - 1):
            markov_chain[words[i]].append(words[i + 1])
    
    return markov_chain

# Step 2: Generate New Titles
def generate_title(chain, length=8):
    word = random.choice(list(chain.keys()))  # Start with a random word
    title = [word]
    
    for _ in range(length - 1):
        if word in chain:
            word = random.choice(chain[word])
            title.append(word)
        else:
            break
    
    return " ".join(title)

# Build chain
markov_chain = build_markov_chain(titles)

# Generate new titles
for _ in range(5):
    print(generate_title(markov_chain))

Worth Doing Badly | Jordan Peterson | Jordan
Should Be A Monster" | Jordan Peterson's Life
7 Harsh Truths To Take Control Of Your
Rules For Life Advice Will Change Your Life
Worth Doing Is Worth Doing Is Worth Doing


Time to generate a text file with the markov chains keywords!

In [ ]:
# Example list of keywords (this would be from your dataset or a predefined list)
keywords = keyword_df_clean["Title"].tolist()

# Step 3: Adjust the Title Generation to Include Keywords
def generate_title_with_keywords(chain, keywords, length=8, max_attempts=100):
    attempts = 0
    while attempts < max_attempts:
        title = generate_title(chain, length)
        if any(keyword in title for keyword in keywords):
            return title
        attempts += 1
    return "No valid title found"

# Generate new titles that include the keywords
with open("generated_titles.txt", "w") as f:
    for _ in range(5):
        generated_title = generate_title_with_keywords(markov_chain, keywords)
        f.write(generated_title + "\n")  # Write each title on a new line

        print(generate_title_with_keywords(markov_chain, keywords))


To Treat Yourself Better | 12 Rules For
Should Be A Monster" | 12 Rules For
Monster" | 12 Rules For Life Advice Will
Only a Few Apply It | 12 Rules
Better | 12 Rules For Life Advice Will


Reading the generated file and splitting it into a list of different lines:

In [1]:
# Open the txt file and read its lines
with open('generated_titles.txt', 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

print(lines)


['Be A Monster" | Jordan Peterson | 12', '12 Rules For Life Advice Will Change Your', 'But Only a Few Apply It | 12', 'Apply It | 12 Rules For Life Advice', 'You Do | 12 Rules For Life Advice']


Time to complete the sentence using GPT2:

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


def complete_sentence(prompt, max_length=50):

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    

    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    

    completed_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return completed_text


sentences = lines

for sentence in sentences:
    completed = complete_sentence(sentence)
    print(f"Input: {sentence}")
    print(f"Output: {completed}")
    print("="*50)


/Users/arnoldm./NLP/venv2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token

Input: Be A Monster" | Jordan Peterson | 12
Output: Be A Monster" | Jordan Peterson | 12/10/2014

"I'm a Monster," Peterson said. "I've been a monster for a long time. I've had a lot of success. But I'm not a big guy


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: 12 Rules For Life Advice Will Change Your
Output: 12 Rules For Life Advice Will Change Your Life

The following are some of the most common questions you'll get when you're trying to figure out what to do with your life.
. . .
 (1) What is your favorite thing


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: But Only a Few Apply It | 12
Output: But Only a Few Apply It | 12.5

The Best of the Best | 11.4
.
,
 (1) The Best Of The Worst | 10.9
:
- The best of all time | 9.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: Apply It | 12 Rules For Life Advice
Output: Apply It | 12 Rules For Life Advice | 13 Tips For Getting A Job | 14 How To Get A Good Job In The U.S. | 15 How to Get Your Job Done | 16 How You Can Get The Best Job Ever | 17 How
Input: You Do | 12 Rules For Life Advice
Output: You Do | 12 Rules For Life Advice

The following are the 12 rules for life advice for people who are struggling with depression.
. . .
 (1) Don't be afraid to ask for help. If you're struggling, you


In [52]:
!pip install transformers datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.8 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 9.7 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.1/32.1 MB 8.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 923.9 kB/s eta 0:00:00B/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2

In [56]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load the dataset
dataset = load_dataset("text", data_files={"train": "generated_titles.txt"})


In [61]:
!pip install 'accelerate>=0.26.0'
!pip install transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 4.6 MB/s eta 0:00:000:00:010:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: no matches found: transformers[torch]
